In [14]:
import numpy as np

In [15]:
from jobflow.managers.local import run_locally

In [16]:
from pyiron_base import Project, job

In [17]:
from python_workflow_definition.pyiron_base import write_workflow_json

In [18]:
from python_workflow_definition.jobflow import load_workflow_json

In [19]:
from quantum_espresso_workflow import (
    calculate_qe as _calculate_qe, 
    generate_structures as _generate_structures, 
    get_bulk_structure as _get_bulk_structure, 
    plot_energy_volume_curve as _plot_energy_volume_curve,
)

In [20]:
workflow_json_filename = "pyiron_base_to_jobflow_qe.json"

In [21]:
calculate_qe = job(_calculate_qe, output_key_lst=["energy", "volume", "structure"])
generate_structures = job(_generate_structures)
plot_energy_volume_curve = job(_plot_energy_volume_curve)
get_bulk_structure = job(_get_bulk_structure)

In [22]:
pseudopotentials = {"Al": "Al.pbe-n-kjpaw_psl.1.0.0.UPF"}

In [23]:
pr = Project("test")
pr.remove_jobs(recursive=True, silently=True)

0it [00:00, ?it/s]

In [24]:
structure = get_bulk_structure(
    element="Al",
    a=4.05,
    cubic=True,
    pyiron_project=pr,
)

In [25]:
calc_mini = calculate_qe(
    working_directory="mini",
    input_dict={
        "structure": structure,
        "pseudopotentials": pseudopotentials,
        "kpts": (3, 3, 3),
        "calculation": "vc-relax",
        "smearing": 0.02,
    },
    pyiron_project=pr,
)

In [26]:
number_of_strains = 5
structure_lst = generate_structures(  # the generate_structures() function is not available in the workflow graph
    structure=calc_mini.output.structure,
    strain_lst=np.linspace(0.9, 1.1, number_of_strains),
    pyiron_project=pr,
    list_length=number_of_strains,
)

In [27]:
job_strain_lst = []
for i, structure_strain in enumerate(structure_lst):
    calc_strain = calculate_qe(
        working_directory="strain_" + str(i),
        input_dict={
            "structure": structure_strain,
            "pseudopotentials": pseudopotentials,
            "kpts": (3, 3, 3),
            "calculation": "scf",
            "smearing": 0.02,
        },
        pyiron_project=pr,
    )
    job_strain_lst.append(calc_strain)

In [28]:
plot = plot_energy_volume_curve(
    volume_lst=[job.output.volume for job in job_strain_lst],
    energy_lst=[job.output.energy for job in job_strain_lst],
    pyiron_project=pr,
)

In [29]:
write_workflow_json(delayed_object=plot, file_name=workflow_json_filename)

In [ ]:
!cat {workflow_json_filename}

In [20]:
flow = load_workflow_json(file_name=workflow_json_filename)

In [21]:
result = run_locally(flow)
result

2025-03-17 09:50:21,792 INFO Started executing jobs locally
2025-03-17 09:50:22,018 INFO Starting job - get_bulk_structure (4c9813a9-b659-4297-91fb-09afce5a7760)
2025-03-17 09:50:22,021 INFO Finished job - get_bulk_structure (4c9813a9-b659-4297-91fb-09afce5a7760)
2025-03-17 09:50:22,022 INFO Starting job - get_dict (3690ec2a-b248-407d-b66f-f851a7f17c0a)
2025-03-17 09:50:22,024 INFO Finished job - get_dict (3690ec2a-b248-407d-b66f-f851a7f17c0a)
2025-03-17 09:50:22,025 INFO Starting job - calculate_qe (af9fbdab-b34c-4e21-a8d5-399dc2d723f6)
2025-03-17 09:50:51,064 INFO Finished job - calculate_qe (af9fbdab-b34c-4e21-a8d5-399dc2d723f6)
2025-03-17 09:50:51,065 INFO Starting job - generate_structures (cf237605-9077-4f81-8e4d-06632fc8e485)
2025-03-17 09:50:51,074 INFO Finished job - generate_structures (cf237605-9077-4f81-8e4d-06632fc8e485)
2025-03-17 09:50:51,080 INFO Starting job - get_dict (ae13ecea-119c-4ca3-be88-0a5664375495)
2025-03-17 09:50:51,090 INFO get_dict failed with exception:
T

{'4c9813a9-b659-4297-91fb-09afce5a7760': {1: Response(output={'numbers': [13, 13, 13, 13], 'positions': [[0.0, 0.0, 0.0], [0.0, 2.025, 2.025], [2.025, 0.0, 2.025], [2.025, 2.025, 0.0]], 'cell': [[4.05, 0.0, 0.0], [0.0, 4.05, 0.0], [0.0, 0.0, 4.05]], 'pbc': [True, True, True]}, detour=None, addition=None, replace=None, stored_data=None, stop_children=False, stop_jobflow=False, job_dir=PosixPath('/home/geiger_j/aiida_projects/adis/git-repos/python-workflow-definition'))},
 '3690ec2a-b248-407d-b66f-f851a7f17c0a': {1: Response(output={'structure': {'numbers': [13, 13, 13, 13], 'positions': [[0.0, 0.0, 0.0], [0.0, 2.025, 2.025], [2.025, 0.0, 2.025], [2.025, 2.025, 0.0]], 'cell': [[4.05, 0.0, 0.0], [0.0, 4.05, 0.0], [0.0, 0.0, 4.05]], 'pbc': [True, True, True]}, 'pseudopotentials': {'Al': 'Al.pbe-n-kjpaw_psl.1.0.0.UPF'}, 'kpts': [3, 3, 3], 'calculation': 'vc-relax', 'smearing': 0.02}, detour=None, addition=None, replace=None, stored_data=None, stop_children=False, stop_jobflow=False, job_dir